# Grammatical Error Correction with OpenVINO
AI-based auto-correction products are becoming increasingly popular due to their ease of use, editing speed, and affordability. These products improve the quality of written text in emails, blogs, and chats.

Grammatical Error Correction (GEC) is the task of correcting different types of errors in text such as spelling, punctuation, grammatical and word choice errors. GEC is typically formulated as a sentence correction task. A GEC system takes a potentially erroneous sentence as input and is expected to transform it into a more correct version. See the example given below:

         Input (Erroneous)	       Output (Corrected)
         
        ------------------         ------------------
        
    I like to rides my bicycle.	I like to ride my bicycle.



# Optimization / Conversion to IR
https://huggingface.co/docs/optimum/index

https://docs.openvino.ai/2024/documentation/openvino-ir-format.html

# How does it work?

A Grammatical Error Correction task can be thought of as a sequence-to-sequence task where a model is trained to take a grammatically incorrect sentence as input and return a grammatically correct sentence as output. We will use the FLAN-T5 model finetuned on an expanded version of the JFLEG dataset. --> https://paperswithcode.com/dataset/jfleg

--> https://aclanthology.org/E17-2037.pdf

The version of FLAN-T5 released with the Scaling Instruction-Finetuned Language Models paper is an enhanced version of T5 that has been finetuned on a combination of tasks. The paper explores instruction finetuning with a particular focus on scaling the number of tasks, scaling the model size, and finetuning on chain-of-thought data. The paper discovers that overall instruction finetuning is a general method that improves the performance and usability of pre-trained language models.

T5 --> Paper https://arxiv.org/pdf/2210.11416

Repo --> https://github.com/google-research/t5x
### Grammatical correctness checking

to reduce the number of sentences required to be processed, you can perform grammatical correctness checking. This task should be considered as a simple binary text classification, where the model gets input text and predicts label 1 if a text contains any grammatical errors and 0 if it does not. You will use the roberta-base-CoLA model, the RoBERTa Base model finetuned on the CoLA dataset.

Roberta paper --> https://arxiv.org/abs/1907.11692

Roberta trained with Cola --> https://huggingface.co/textattack/roberta-base-CoLA

Cola Dataset --> https://huggingface.co/datasets/shivkumarganesh/CoLA

In [1]:
#%pip install -q "torch>=2.1.0" "git+https://github.com/huggingface/optimum-intel.git" "openvino>=2024.0.0" "onnx<1.16.2" tqdm "gradio>=4.19" "transformers>=4.33.0" --extra-index-url https://download.pytorch.org/whl/cpu
%pip install -q "nncf>=2.9.0" datasets jiwer

Note: you may need to restart the kernel to use updated packages.


In [2]:
from pathlib import Path
from transformers import pipeline, AutoTokenizer
from optimum.intel.openvino import OVModelForSeq2SeqLM, OVModelForSequenceClassification

In [3]:
from notebook_utils import device_widget

device = device_widget()

device

Dropdown(description='Device:', index=1, options=('CPU', 'AUTO'), value='AUTO')

# Download and Convert Models
Optimum Intel can be used to load optimized models from the Hugging Face Hub and create pipelines to run an inference with OpenVINO Runtime using Hugging Face APIs. The Optimum Inference models are API compatible with Hugging Face Transformers models. This means we just need to replace AutoModelForXxx class with the corresponding OVModelForXxx class.

# Grammar Checker

In [4]:

grammar_checker_model_id = "textattack/roberta-base-CoLA"
grammar_checker_dir = Path("roberta-base-cola")
grammar_checker_tokenizer = AutoTokenizer.from_pretrained(grammar_checker_model_id)

if grammar_checker_dir.exists():
    grammar_checker_model = OVModelForSequenceClassification.from_pretrained(grammar_checker_dir, device=device.value)
else:
    grammar_checker_model = OVModelForSequenceClassification.from_pretrained(grammar_checker_model_id, export=True, device=device.value, load_in_8bit=False)
    grammar_checker_model.save_pretrained(grammar_checker_dir)

Compiling the model to AUTO ...


In [5]:
input_text = "The car is running with electriciti"
grammar_checker_pipe = pipeline(
    "text-classification",
    model=grammar_checker_model,
    tokenizer=grammar_checker_tokenizer,
)
result = grammar_checker_pipe(input_text)[0]
print(f"input text: {input_text}")
print(f'predicted label: {"contains_errors" if result["label"] == "LABEL_1" else "no errors"}')
print(f'predicted score: {result["score"] :.2}')

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


input text: The car is running with electriciti
predicted label: contains_errors
predicted score: 0.79


In [6]:
result

{'label': 'LABEL_1', 'score': 0.7926265597343445}

# Grammar Corrector

In [7]:
grammar_corrector_model_id = "pszemraj/flan-t5-large-grammar-synthesis"
grammar_corrector_dir = Path("flan-t5-large-grammar-synthesis")
grammar_corrector_tokenizer = AutoTokenizer.from_pretrained(grammar_corrector_model_id)

if grammar_corrector_dir.exists():
    grammar_corrector_model = OVModelForSeq2SeqLM.from_pretrained(grammar_corrector_dir, device=device.value)
else:
    grammar_corrector_model = OVModelForSeq2SeqLM.from_pretrained(grammar_corrector_model_id, export=True, device=device.value)
    grammar_corrector_model.save_pretrained(grammar_corrector_dir)

Compiling the encoder to AUTO ...
Compiling the decoder to AUTO ...
Compiling the decoder to AUTO ...


In [8]:
grammar_corrector_pipe = pipeline(
    "text2text-generation",
    model=grammar_corrector_model,
    tokenizer=grammar_corrector_tokenizer,
)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [9]:
result = grammar_corrector_pipe(input_text)[0]
print(f"input text:     {input_text}")
print(f'generated text: {result["generated_text"]}')

input text:     The car is running with electriciti
generated text: The car is running on electricity.


# GEC Pipeline
1. Split text on sentences.
2. Check grammatical correctness for each sentence using Grammar Checker.
3. Generate an improved version of the sentence if required.

In [10]:
import re
import transformers
from tqdm.notebook import tqdm


def split_text(text: str) -> list:
    """
    Split a string of text into a list of sentence batches.

    Parameters:
    text (str): The text to be split into sentence batches.

    Returns:
    list: A list of sentence batches. Each sentence batch is a list of sentences.
    """
    # Split the text into sentences using regex
    sentences = re.split(r"(?<=[^A-Z].[.?]) +(?=[A-Z])", text)
    #sentences = text.split(".")

    # Initialize a list to store the sentence batches
    sentence_batches = []

    # Initialize a temporary list to store the current batch of sentences
    temp_batch = []

    # Iterate through the sentences
    for sentence in sentences:
        # Add the sentence to the temporary batch
        temp_batch.append(sentence)

        # If the length of the temporary batch is between 2 and 3 sentences, or if it is the last batch, add it to the list of sentence batches
        if len(temp_batch) >= 2 and len(temp_batch) <= 3 or sentence == sentences[-1]:
            sentence_batches.append(temp_batch)
            temp_batch = []

    return sentence_batches


def correct_text(
    text: str,
    checker: transformers.pipelines.Pipeline,
    corrector: transformers.pipelines.Pipeline,
    separator: str = " ",
) -> str:
    """
    Correct the grammar in a string of text using a text-classification and text-generation pipeline.

    Parameters:
    text (str): The inpur text to be corrected.
    checker (transformers.pipelines.Pipeline): The text-classification pipeline to use for checking the grammar quality of the text.
    corrector (transformers.pipelines.Pipeline): The text-generation pipeline to use for correcting the text.
    separator (str, optional): The separator to use when joining the corrected text into a single string. Default is a space character.

    Returns:
    str: The corrected text.
    """
    # Split the text into sentence batches
    sentence_batches = split_text(text)

    # Initialize a list to store the corrected text
    corrected_text = []

    # Iterate through the sentence batches
    for batch in tqdm(sentence_batches, total=len(sentence_batches), desc="correcting text.."):
        # Join the sentences in the batch into a single string
        raw_text = " ".join(batch)


        # Check the grammar quality of the text using the text-classification pipeline
        results = checker(raw_text)

        # Only correct the text if the results of the text-classification are not LABEL_1 or are LABEL_1 with a score below 0.9
        if results[0]["label"] != "LABEL_1" or (results[0]["label"] == "LABEL_1" and results[0]["score"] < 0.9):
            # Correct the text using the text-generation pipeline
            corrected_batch = corrector(raw_text)
            corrected_text.append(corrected_batch[0]["generated_text"])
        else:
            corrected_text.append(raw_text)

    # Join the corrected text into a single string
    corrected_text = separator.join(corrected_text)

    return corrected_text

In [11]:
import datetime

In [12]:
default_text = (
    """Most of the course is about semantic or  content of language but there are also interesting
    topics to be learned from the servicefeatures except statistics in characters in documents. At
    this point, He introduces herself as his native English speaker and goes on to say that if
    you contine to work on social scnce"""
)
time1 = datetime.datetime.now()
corrected_text = correct_text(default_text, grammar_checker_pipe, grammar_corrector_pipe)
time2 = datetime.datetime.now()
seconds = (time2-time1).seconds
print(f"Time in seconds {seconds}")

correcting text..:   0%|          | 0/1 [00:00<?, ?it/s]

Time in seconds 13


In [13]:
print(f"input text:     {default_text}\n")
print(f"generated text: {corrected_text}")

input text:     Most of the course is about semantic or  content of language but there are also interesting
    topics to be learned from the servicefeatures except statistics in characters in documents. At
    this point, He introduces herself as his native English speaker and goes on to say that if
    you contine to work on social scnce

generated text: Most of the course is about the semantic content of language but there are also interesting topics to be learned from the service features except statistics in characters in documents. At this point, she introduces herself as a native English speaker and goes on to say that if you continue to work on social science, you will continue to be successful.


# Quantization
NNCF enables post-training quantization by adding quantization layers into model graph and then using a subset of the training dataset to initialize the parameters of these additional quantization layers. Quantized operations are executed in INT8 instead of FP32/FP16 making model inference faster.

https://github.com/openvinotoolkit/nncf/

Grammar checker model takes up a tiny portion of the whole text correction pipeline so we optimize only the grammar corrector model. Grammar corrector itself consists of three models: encoder, first call decoder and decoder with past. The last model's share of inference dominates the other ones. Because of this we quantize only it.

The optimization process contains the following steps:

1. Create a calibration dataset for quantization.
2. Run nncf.quantize() to obtain quantized models.
3. Serialize the INT8 model using openvino.save_model() function.


In [14]:
from notebook_utils import quantization_widget
import openvino as ov
to_quantize = quantization_widget()

to_quantize

Checkbox(value=True, description='Quantization')

In [15]:
core = ov.Core()

In [16]:
from src.utils import get_quantized_pipeline, CALIBRATION_DATASET_SIZE

grammar_corrector_pipe_fp32 = grammar_corrector_pipe
grammar_corrector_pipe_int8 = None
if to_quantize.value:
    quantized_model_path = Path("quantized_decoder_with_past") / "openvino_model.xml"
    grammar_corrector_pipe_int8 = get_quantized_pipeline(
        grammar_corrector_pipe_fp32,
        grammar_corrector_tokenizer,
        core,
        grammar_corrector_dir,
        quantized_model_path,
        device.value,
        calibration_dataset_size=100,
    )

Output()

Output()

INFO:nncf:72 ignored nodes were found by names in the NNCFGraph
INFO:nncf:145 ignored nodes were found by names in the NNCFGraph


Output()

Compiling the encoder to AUTO ...
Compiling the decoder to AUTO ...
Compiling the decoder to AUTO ...
Compiling the decoder to AUTO ...
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [17]:
if to_quantize.value:
    time1 = datetime.datetime.now()
    corrected_text_int8 = correct_text(default_text, grammar_checker_pipe, grammar_corrector_pipe_int8)
    time2 = datetime.datetime.now()
    seconds = (time2-time1).seconds
    print(f"Time in seconds {seconds}")
    print(f"Input text:                   {default_text}\n")
    print(f"Generated text by INT8 model: {corrected_text_int8}")

correcting text..:   0%|          | 0/1 [00:00<?, ?it/s]

Time in seconds 12
Input text:                   Most of the course is about semantic or  content of language but there are also interesting
    topics to be learned from the servicefeatures except statistics in characters in documents. At
    this point, He introduces herself as his native English speaker and goes on to say that if
    you contine to work on social scnce

Generated text by INT8 model: Most of the course is about the semantic content of language but there are also interesting topics to be learned from the service features except statistics in characters in documents. At this point, she introduces herself as a native English speaker and goes on to say that if you continue to work on social science, you will continue to be successful.


In [18]:
from src.utils import calculate_compression_rate

if to_quantize.value:
    model_size_fp32, model_size_int8 = calculate_compression_rate(
        grammar_corrector_dir / "openvino_decoder_with_past_model.xml",
        quantized_model_path,
    )

Model footprint comparison:
    * FP32 IR model size: 1658150.20 KB
    * INT8 IR model size: 416957.99 KB


In [19]:
from src.utils import calculate_inference_time_and_accuracy

TEST_SUBSET_SIZE = 50

if to_quantize.value:
    inference_time_fp32, accuracy_fp32 = calculate_inference_time_and_accuracy(grammar_corrector_pipe_fp32, TEST_SUBSET_SIZE)
    print(f"Evaluation results of FP32 grammar correction pipeline. Accuracy: {accuracy_fp32:.2f}%. Time: {inference_time_fp32:.2f} sec.")
    inference_time_int8, accuracy_int8 = calculate_inference_time_and_accuracy(grammar_corrector_pipe_int8, TEST_SUBSET_SIZE)
    print(f"Evaluation results of INT8 grammar correction pipeline. Accuracy: {accuracy_int8:.2f}%. Time: {inference_time_int8:.2f} sec.")
    print(f"Performance speedup: {inference_time_fp32 / inference_time_int8:.3f}")
    print(f"Accuracy drop :{accuracy_fp32 - accuracy_int8:.2f}%.")
    print(f"Model footprint reduction: {model_size_fp32 / model_size_int8:.3f}")

Evaluation:   0%|          | 0/50 [00:00<?, ?it/s]

Evaluation results of FP32 grammar correction pipeline. Accuracy: 58.04%. Time: 187.64 sec.


Evaluation:   0%|          | 0/50 [00:00<?, ?it/s]

Evaluation results of INT8 grammar correction pipeline. Accuracy: 57.89%. Time: 155.55 sec.
Performance speedup: 1.206
Accuracy drop :0.15%.
Model footprint reduction: 3.977
